### SVM Hard Margin

#### Exercise 1. Breast Cancer

##### 1. Read data

In [6]:
from sklearn import datasets

cancer_data = datasets.load_breast_cancer()
# show to test record 5th
print(cancer_data.data[5])
print(cancer_data.data.shape)
#target set
print(cancer_data.target)

[1.245e+01 1.570e+01 8.257e+01 4.771e+02 1.278e-01 1.700e-01 1.578e-01
 8.089e-02 2.087e-01 7.613e-02 3.345e-01 8.902e-01 2.217e+00 2.719e+01
 7.510e-03 3.345e-02 3.672e-02 1.137e-02 2.165e-02 5.082e-03 1.547e+01
 2.375e+01 1.034e+02 7.416e+02 1.791e-01 5.249e-01 5.355e-01 1.741e-01
 3.985e-01 1.244e-01]
(569, 30)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 

In [7]:
y = cancer_data.target
y[cancer_data.target == 0] = -1

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancer_data.data, y, test_size=0.3,random_state=109)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(398, 30)
(171, 30)
(398,)
(171,)


##### 2. Classification

method 1. quadratic programming

In [9]:
from cvxopt import matrix, solvers
import numpy as np

# build P ~ K
X0 = X_train[y_train == 1]
X1 = X_train[y_train == -1]
N = len(X_train)
V = np.concatenate((X0.T, -X1.T), axis = 1)

P = matrix(V.T.dot(V)) # P ~ K in slide see definition of V, K near eq (8)
q = matrix(-np.ones((N, 1))) # all-one vector

# build A, b, G, h
G = matrix(-np.eye(N)) # for all lambda_n >= 0! note that we solve -g(lambda) -> min
h = matrix(np.zeros((N, 1)))
A = matrix(y_train.reshape((-1, N))) # the equality constrain is actually y^T lambda = 0
A = A * 1.0
b = matrix(np.zeros((1, 1)))
solvers.options['show_progress'] = False
sol = solvers.qp(P, q, G, h, A, b)
l = np.array(sol['x']) # lambda
print('lambda = ')
print(l.T)

lambda = 
[[3.56089995e-176 1.24980218e-175 1.93923233e-176 1.61784321e-173
  3.00771794e-176 8.18302662e-175 1.65274665e+004 6.34457891e-176
  3.92173179e-176 2.69839567e-175 4.25940551e-176 5.16505691e-176
  5.51505131e-176 3.43407719e-176 3.72034455e-176 4.63632770e-176
  1.65866694e+002 2.72180536e-176 5.37271599e-176 3.58101659e-176
  9.54232649e-176 2.08152836e-176 4.13542899e+003 7.75213092e-176
  2.00970466e-176 6.59378040e-176 7.08861621e-176 2.94456426e-176
  4.06473512e-176 1.01181157e-175 3.79407884e-176 4.65329236e-176
  8.56236594e-176 3.96026687e-176 8.84644844e+004 2.89056692e-176
  2.92345642e-176 3.31836673e-176 3.16820351e-176 6.59730905e-176
  6.94845426e-176 2.82811428e-176 6.86749228e-176 1.65623752e-175
  8.08559069e-176 8.80714632e-176 3.54817083e-176 2.94355384e-176
  5.58674485e-176 2.46786614e-176 2.80707551e-176 5.96032214e-176
  2.88980594e-176 1.93475741e-175 1.39756844e-175 2.44348998e-176
  2.08006497e-174 5.37951877e-176 6.28855588e-176 4.82993172e-176


In [10]:
# compute coefficients
epsilon = 1e-5
S = np.where(l > epsilon)[0]
VS = V[:, S]
XS = X_train.T[:, S]
yS = y_train[S]
lS = l[S]
# calculate w and b
w = VS.dot(lS)
w0 = np.mean(yS.T - w.T.dot(XS))
print('W = ', w.T)
print('W0 = ', w0)

W =  [[ 2.04803509e+01 -9.89367070e-01 -1.87123830e+00 -2.81760618e-02
  -1.55737690e+02  8.87791005e+01 -3.33023883e+01 -1.58515451e+02
   2.01325885e+02 -2.16477957e+02 -1.89074655e+01  4.66128490e+00
  -7.57110843e+00  4.24399585e-02  9.38252656e+01 -3.69208238e+02
   4.00605557e+02 -2.19625774e+02  1.85128470e+02 -1.86567763e+01
   2.31400020e-02 -6.09657897e-01  7.91790183e-01 -1.13547400e-01
   9.77405245e+00  1.00007750e+02 -7.71757724e+01 -1.70127871e+02
  -1.47767933e+02  7.94037229e+01]]
W0 =  44.78540092927298


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
# predict on test set
y_pred = np.sign(w.T.dot(X_test.T) + w0)

# accuracy
print(accuracy_score(y_test, y_pred[0]))
print(precision_score(y_test, y_pred[0]))
print(recall_score(y_test, y_pred[0]))
print(confusion_matrix(y_test, y_pred[0]))

0.8538011695906432
0.8120300751879699
1.0
[[ 38  25]
 [  0 108]]


method 2. sklearn

In [13]:
from sklearn.svm import SVC

clf = SVC(kernel = 'linear', C = 1e5) # choose a big number for Hard Margin
clf.fit(X_train, y_train)
w = clf.coef_
w0 = clf.intercept_
print('w = ', w)
print('w0 = ', w0)

w =  [[ 1.55841303e+04  5.69829895e+03  6.01236573e+02 -2.11652062e+02
  -7.58418989e+03 -3.58287335e+03 -2.28082132e+04 -1.67774049e+04
  -2.14646639e+04  2.35627498e+02 -4.23531681e+03  3.41852567e+04
  -7.07060988e+02 -1.82451703e+03 -1.29485985e+03  5.13476398e+03
  -1.07869294e+03 -3.52075548e+03 -5.84510533e+03  3.23810407e+02
   1.32407387e+03 -8.43438202e+03 -6.60376556e+02 -6.59702879e+00
  -1.27800564e+04  1.10353669e+04 -4.11879410e+04 -3.48086330e+04
  -6.38834076e+04  8.62849567e+02]]
w0 =  [88059.70921289]


In [14]:
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.9649122807017544
0.9811320754716981
0.9629629629629629
[[ 61   2]
 [  4 104]]


### SVM Soft Margin

#### Exercise 2. Sonar Signals

1. read and preprocess data

In [15]:
# read data from file
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('D:/python/pyforml/week06_perceptron/sonar.all-data.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [16]:
# convert output from string {'M', 'R'} into  {1, -1}
y_df = df[60]
targets_label = {'M': 1.0, 'R': -1.0}
targets_df = np.array([targets_label[item] for item in y_df])
print(targets_df.shape)

(208,)


In [17]:
# prepare data
X = df.drop(60, axis=1)
X.shape

(208, 60)

2. split data into training and test set

In [18]:
# split data into training-test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, targets_df, test_size=0.30, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(145, 60)
(63, 60)
(145,)
(63,)


3. classification

method 1. quadratic programming

In [34]:
from cvxopt import matrix, solvers

C = 200
N = len(X_train)
X0 = X_train[y_train == 1]
X1 = X_train[y_train == -1]
# build K
V = np.concatenate((X0.T, -X1.T), axis = 1)
K = matrix(V.T.dot(V))

p = matrix(-np.ones((N, 1)))
# build A, b, G, h
G = matrix(np.vstack((-np.eye(N), np.eye(N))))

h = matrix(np.vstack((np.zeros((N, 1)), C*np.ones((N, 1)))))
A = matrix(y_train.reshape((-1, N)))
b = matrix(np.zeros((1, 1)))
solvers.options['show_progress'] = False
sol = solvers.qp(K, p, G, h, A, b)

l = np.array(sol['x'])
print('lambda = \n', l.T)

lambda = 
 [[3.03036624e-07 4.96323329e+01 1.94805353e+02 1.99999999e+02
  8.13833270e+00 8.10031043e-07 4.35238456e+01 5.31620759e+01
  5.98945884e-07 5.61567008e+00 1.99999996e+02 4.84416161e-07
  1.99322257e+02 4.59827859e-07 1.34902676e-06 8.88661206e+01
  5.38827346e-07 2.66894023e+01 7.60547546e-07 1.99999999e+02
  3.74854606e-06 5.62639292e-07 5.00258249e-06 1.65145215e+02
  1.45552471e-06 1.42062781e-06 1.99999996e+02 4.60248206e-06
  1.26370253e-06 7.11739676e-07 6.50394684e-07 4.57838958e-07
  1.99999999e+02 3.88025256e-06 5.03814613e-07 1.37572313e+02
  4.77635901e-07 1.99999999e+02 4.70160556e-07 6.04610853e-07
  2.28568067e-06 9.25147078e-07 3.61847048e-07 4.69432605e-07
  1.25892365e+02 5.58295811e+01 5.52314050e+01 3.53511249e-07
  1.26819975e+02 1.73952006e-06 1.67133768e-06 4.64066843e-07
  6.25485657e+01 8.76621866e-07 1.99999988e+02 1.99999999e+02
  2.32513840e-07 1.19890533e-06 1.99999979e+02 5.22135855e+01
  4.37087607e-07 3.53764722e-07 6.10418933e-06 4.37080819e-

In [35]:
S = np.where(l > 1e-5)[0] # support set
S2 = np.where(l < .999*C)[0]

M = np.array([val for val in S if val in S2]) # intersection of two lists

XT = np.array(X_train) # we need each column to be one data point in this alg
VS = V[:, S]
lS = l[S]
yM = np.array(y_train)[M]

XM = XT[M]

w_dual = VS.dot(lS).reshape(-1, 1)
b_dual = np.mean(yM.T - w_dual.T.dot(XM.T))
print(w_dual.T, b_dual)

[[ 21.2135181   -6.73090046 -15.35460532  15.05533195   9.95861111
    1.85649896 -11.03861506  -4.83267984   7.34511454  -5.73492904
   10.31726354   2.71788542  -0.22614245   1.10082845  -0.16470185
   -3.1963187   -3.69721201   2.77285179   1.06265363   0.87956018
   -5.17912571   4.58010346  -2.68459607   3.93988181  -0.78427927
   -1.49247935  -0.5522061    1.89789989  -5.14355276  11.64876853
  -14.73916492   2.818122     3.88958285  -4.86963981   2.7623044
   -7.4797356    1.53509606  -2.55579627   4.32771034  -2.50650008
   -3.71372346  -3.70053939   9.88374578  -0.18490579  -2.22343833
   11.48878863  -3.26009388   4.72489687  17.10250302 -16.30146613
    2.13239839   2.69342789   1.42587907   6.18977237  -1.95498578
   -8.10311857  -6.85604425   3.28194714   5.29251941   2.73464288]] -0.005792603885938371


In [36]:
# predict
pred = np.sign(w_dual.T.dot(X_test.T) + b_dual)

# accuracy
print(accuracy_score(y_test, pred[0]))
print(precision_score(y_test, pred[0]))
print(recall_score(y_test, pred[0]))
print(confusion_matrix(y_test, pred[0]))

0.7619047619047619
0.7941176470588235
0.7714285714285715
[[21  7]
 [ 8 27]]


method 2. sklearn

In [37]:
from sklearn.svm import SVC
clf = SVC(kernel = 'linear', C = 200) # use the same C
clf.fit(X_train, y_train)

w = clf.coef_
w0 = clf.intercept_
print('w = ', w)
print('b = ', w0)

w =  [[ 18.86249098  -6.33870903 -13.13013723  15.19727802   6.94389109
    1.3707947   -7.97911488  -5.39122567   7.0942688   -9.2205407
   18.60588768  -0.63108361  -0.86118687  -1.38102747   1.21354338
   -2.3738622   -4.07985943   5.10306906  -2.08121323   4.51356674
   -5.27767426   3.42777057  -3.28908432   7.56856304  -0.30469426
   -2.34982934   2.50157828  -1.13341559  -1.58462955   8.60618559
  -13.55535164   4.8193884    4.27312485  -4.48153437   3.73957584
   -7.28302251  -0.67727654   1.15149532   5.03188987  -6.17312616
    0.55096319  -5.17864406  10.0486221    0.40897097  -1.72127855
    6.11073173   2.09858325  12.38589849  12.18324146 -14.90467131
    5.95039622   4.16948956   0.47477646  10.35026853   0.25557821
   -6.67226513  -8.88895183   4.57111583   6.97917082   4.97822679]]
b =  [-6.44642235]


In [38]:
# predict
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.7777777777777778
0.8
0.8
[[21  7]
 [ 7 28]]


#### Exercise 3. Spam Filtering

In [ ]:
'''
57 (continuous) attributes + 1 (nominal) indicator

0-47: percentage [0-100] of certain common words
48-53: percentage [0-100] of certain common characters - ;([!$#
54: average length [1-...] of uninterrupted sequences of capital letters
55: length [1-...] of longest uninterrupted sequences of capital letters
56: sum of length [1-...] of uninterrupted sequences of capital lettes = total number of capital letters
57: class {0,1} denotes whether a email was considered spam (1) or not (0)
'''

1. Read and preprocess data

In [2]:
import pandas as pd

df = pd.read_csv('./spambase/spambase.data', header=None)
print(df.shape)
df.head()

(4601, 58)


,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [13]:
# renaming the columns
df.columns  = ["word_freq_make", "word_freq_address", "word_freq_all", "word_freq_3d", 
                      "word_freq_our", "word_freq_over", "word_freq_remove", "word_freq_internet", 
                      "word_freq_order", "word_freq_mail", "word_freq_receive", "word_freq_will", 
                      "word_freq_people", "word_freq_report", "word_freq_addresses", "word_freq_free", 
                      "word_freq_business", "word_freq_email", "word_freq_you", "word_freq_credit", 
                      "word_freq_your", "word_freq_font", "word_freq_000", "word_freq_money", "word_freq_hp", 
                      "word_freq_hpl", "word_freq_george", "word_freq_650", "word_freq_lab", "word_freq_labs", 
                      "word_freq_telnet", "word_freq_857", "word_freq_data", "word_freq_415", "word_freq_85", 
                      "word_freq_technology", "word_freq_1999", "word_freq_parts", "word_freq_pm", "word_freq_direct",
                      "word_freq_cs", "word_freq_meeting", "word_freq_original", "word_freq_project", "word_freq_re", 
                      "word_freq_edu", "word_freq_table", "word_freq_conference", "char_freq_;", "char_freq_(", 
                      "char_freq_[", "char_freq_!", "char_freq_$", "char_freq_hash", "capital_run_length_average", 
                      "capital_run_length_longest", "capital_run_length_total", "spam"]
df.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_hash,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [4]:
import numpy as np

# convert label to 1/0
label_df = df[57]
y_label = {1: 1.0, 0: -1.0}
y = np.array([y_label[item] for item in label_df])
print(y.shape)

(4601,)


In [14]:
df.describe()

# notice that the last 3 attributes (capital_run) 
# have much higher means and std values than others, 
# which need to be rescaling for better perfomance

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_hash,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [21]:
from sklearn.preprocessing import StandardScaler
# rescaling (standardize)
X = df.drop('spam', axis=1)
scaler = StandardScaler()
X_norm = pd.DataFrame(scaler.fit_transform(X))
X_norm.head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,-0.342434,0.330885,0.712859,-0.0469,0.011565,-0.350266,-0.291794,-0.262562,-0.323302,-0.371364,...,-0.111546,-0.158453,-0.514307,-0.155198,0.624007,-0.308355,-0.103048,-0.045247,0.045298,-0.008724
1,0.345359,0.051909,0.435130,-0.0469,-0.256117,0.672399,0.244743,-0.088010,-0.323302,1.086711,...,-0.111546,-0.158453,-0.026007,-0.155198,0.126203,0.423783,0.008763,-0.002443,0.250563,1.228324
2,-0.145921,-0.165072,0.851723,-0.0469,1.364846,0.343685,0.193644,0.036670,1.974017,0.016422,...,-0.111546,-0.117376,0.014684,-0.155198,0.008496,0.440053,-0.079754,0.145921,2.221106,3.258733
3,-0.342434,-0.165072,-0.556761,-0.0469,0.472573,-0.350266,0.500237,1.308402,0.789462,0.605857,...,-0.111546,-0.158453,-0.007511,-0.155198,-0.161934,-0.308355,-0.103048,-0.052150,-0.062466,-0.152222
4,-0.342434,-0.165072,-0.556761,-0.0469,0.472573,-0.350266,0.500237,1.308402,0.789462,0.605857,...,-0.111546,-0.158453,-0.014910,-0.155198,-0.164387,-0.308355,-0.103048,-0.052150,-0.062466,-0.152222


2. Split data with train:set ratio = 7:3 

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, stratify=y, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3220, 57)
(1381, 57)
(3220,)
(1381,)


3. Classification

In [23]:
from cvxopt import matrix, solvers

C = 200
N = len(X_train)
X0 = X_train[y_train == 1]
X1 = X_train[y_train == -1]
# build K
V = np.concatenate((X0.T, -X1.T), axis = 1)
K = matrix(V.T.dot(V))

p = matrix(-np.ones((N, 1)))
# build A, b, G, h
G = matrix(np.vstack((-np.eye(N), np.eye(N))))

h = matrix(np.vstack((np.zeros((N, 1)), C*np.ones((N, 1)))))
A = matrix(y_train.reshape((-1, N)))
b = matrix(np.zeros((1, 1)))
solvers.options['show_progress'] = False
sol = solvers.qp(K, p, G, h, A, b)

l = np.array(sol['x'])
print('lambda = \n', l.T)

lambda = 
 [[1.99999990e+02 4.07448253e-06 2.29775355e-06 ... 4.58007776e-06
  4.83391893e-06 3.88943753e-06]]


In [24]:
S = np.where(l > 1e-5)[0] # support set
S2 = np.where(l < .999*C)[0]

M = np.array([val for val in S if val in S2]) # intersection of two lists

XT = np.array(X_train) # we need each column to be one data point in this alg
VS = V[:, S]
lS = l[S]
yM = np.array(y_train)[M]

XM = XT[M]

w_dual = VS.dot(lS).reshape(-1, 1)
b_dual = np.mean(yM.T - w_dual.T.dot(XM.T))
print(w_dual.T, b_dual)

[[-1.65563554e-01 -6.48861284e-03  7.69230355e-03  1.56072032e+01
   3.19699442e-01  1.52918564e-01  5.76108219e-01  1.63803649e-01
   1.51303815e-01  1.70930042e-01 -2.63188033e-03 -1.44024122e-01
  -1.40660207e-02 -1.87632265e-02  4.59281668e-01  5.50658706e-01
   2.84693954e-01  6.55334334e-02  6.58194532e-02  1.16468120e-01
   1.58100155e-01  1.22379196e-01  8.10645318e-01  1.74119323e-01
  -1.24534441e+00 -5.93489254e-01 -2.20023106e+00  1.20679817e-01
  -1.18470173e-01  7.12094092e-02 -7.46557767e-01  2.30333153e+00
  -1.41845355e-01 -2.27675056e+00 -4.29518896e-01  2.23463008e-01
  -1.62850467e-01 -6.53502523e-02 -6.51184948e-02 -1.32336253e-01
  -5.87465908e-01 -4.86507853e-01 -1.18805280e-01 -3.29124683e-01
  -4.27356692e-01 -8.08125618e-01 -4.30228597e-02 -2.02445288e-01
  -1.65913386e-01 -2.33595512e-02 -1.17993337e-01  4.07987149e-01
   9.31829326e-01  4.85183136e-01  3.00107508e+00  7.95546277e-01
   1.32563477e-01]] 0.19591084948367637


In [25]:
# predict
pred = np.sign(w_dual.T.dot(X_test.T) + b_dual)

# accuracy
print(accuracy_score(y_test, pred[0]))
print(precision_score(y_test, pred[0]))
print(recall_score(y_test, pred[0]))
print(confusion_matrix(y_test, pred[0]))

0.9232440260680667
0.8938848920863309
0.9136029411764706
[[778  59]
 [ 47 497]]


In [26]:
from sklearn.svm import SVC
clf = SVC(kernel = 'linear', C = 200) # use the same C
clf.fit(X_train, y_train)

w = clf.coef_
w0 = clf.intercept_
print('w = ', w)
print('b = ', w0)

w =  [[-1.26769833e-01  1.37072311e-03  4.57469219e-04  1.68596751e+00
   3.50351126e-01  2.22846656e-01  5.10239102e-01  1.87406261e-01
   1.87368329e-01  2.00750939e-01 -5.67650507e-02 -1.38557747e-01
  -2.83618019e-02 -7.81387477e-03  6.70513759e-01  5.61984762e-01
   2.84277327e-01  8.42154094e-02  4.34032528e-02  8.62429008e-02
   1.81592279e-01  1.33905506e-01  4.82671650e-01  1.00449666e-01
  -1.86789367e+00 -5.69244900e-01 -1.85806806e+01  1.34314320e-01
  -1.01581128e+00  3.70661919e-02 -2.89804623e-01  6.46858707e-01
  -1.37235727e-01 -5.28167111e+00 -2.24727888e+00  2.65063549e-01
  -1.37134453e-01 -7.30772145e-02 -8.18871710e-02  1.40683453e-01
  -4.49727712e+00 -9.59264566e-01 -2.86062608e-01 -4.18724145e-01
  -5.22241975e-01 -1.31399926e+00 -3.91439562e-03 -5.98066088e-01
  -1.77692571e-01 -4.58306905e-02  2.85982741e-04  3.28362104e-01
   9.25330244e-01  3.27711218e-01  2.44912883e+00  1.17171130e+00
   2.03657087e-01]]
b =  [-6.34976843]


In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
# predict
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.9275887038377987
0.914179104477612
0.9007352941176471
[[791  46]
 [ 54 490]]
